# Creating the dataframe containing all subjects and all info

### Importing all necessary modules and load the dataset

In [1]:
import os
import pandas as pd
import numpy as np

from bids import BIDSLayout

## Loading all subject relevant info

In [2]:
# load all the BIDS-compatible data with BIDSLayout
fmriprep_dir = '/media/maria/Elements/Bipolife_BIDS/A1/derivatives/fmriprep'
layout = BIDSLayout(fmriprep_dir, validate=False) # validation = false cause fmriprep ouputs are just BIDS-like

# get subject list
sub_idx = layout.get_subjects()

/home/maria/miniconda3/lib/python3.8/site-packages/bids/layout/models.py:148: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  warnings.warn("The 'extension' entity currently excludes the leading dot ('.'). "


In [3]:
excelfile= '/media/maria/Elements/Bipolife_BIDS/mastertable_rename_convert_0903.xls'
hc_df = pd.read_excel(excelfile, sheet_name="control")
nr_df = pd.read_excel(excelfile, sheet_name="no_risk")
lr_df = pd.read_excel(excelfile, sheet_name="low_risk")
hr_df = pd.read_excel(excelfile, sheet_name="high_risk")
bd_df = pd.read_excel(excelfile, sheet_name="bipolar")

# get a subject list for every group
sub_hc = hc_df['BIDS'].tolist()
sub_nr = nr_df['BIDS'].tolist()
sub_lr = lr_df['BIDS'].tolist()
sub_hr = hr_df['BIDS'].tolist()
sub_bd = bd_df['BIDS'].tolist()

In [4]:
sub_hc_info = hc_df[['BIDS','sex', 'age', 'ymrs', 'mrt_idsc']]
sub_nr_info = nr_df[['BIDS','sex', 'age', 'ymrs', 'mrt_idsc']]
sub_lr_info = lr_df[['BIDS','sex', 'age', 'ymrs', 'mrt_idsc']]
sub_hr_info = hr_df[['BIDS','sex', 'age', 'ymrs', 'mrt_idsc']]
sub_bd_info = bd_df[['BIDS','sex', 'age', 'ymrs', 'mrt_idsc']]

In [5]:
# excluded participants 
excluded_df = pd.read_excel(excelfile, sheet_name="excluded")
sub_excluded = excluded_df['BIDS'].tolist()

## Creating the frame of the dataframe based on one subject 

In [10]:
# load one file of the dataset 
sub_fc = pd.read_csv('/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-partial_correlation/sub-DD01/2903/sub-DD01_atlas-difumo_kind-partial_correlation_connectivity_matrix_2903.csv')

In [ ]:
# print the resulting dataframe to get an overview
#sub_fc

In [11]:
column_list= []
for roi1 in sub_fc.columns[1:]: # ignore Unnamed --> [1,:]
    
    # take all values (just one time) out of the row 
    for value in sub_fc[f'{roi1}']:
        
        df_col = []
        
        # stop if the value is one because you just want the upper part of the matrix
        if value ==1:
            break 
       
        else:  # otherwise we want the column name
            
            # get the name of the other ROI to create column name
            idx = sub_fc.index[sub_fc[f'{roi1}'] == value]
            
            roi2 = sub_fc['Unnamed: 0'][idx].to_string()
            roi2 = roi2[5:]
            
            # getrid of any spaces in the naming and replace them with underscores -> better readable 
            roi2 = roi2.replace(' ','_')    
            roi1 = roi1.replace(' ','_')
            
            column_name = f'{roi1}_X_{roi2}'
            column_list.append(column_name)
            
            # change the underscores back to spaces (after saving the modified column name) so that we work with the original names during the whole loop
            roi1 = roi1.replace('_',' ')
            
df_corr = pd.DataFrame(columns = column_list)

In [12]:
# create Dataframe with right column names from one subject 
df = pd.DataFrame()

# add basic info
for column in ['sub', 'scanner', 'group']:
    df[column] = []
# add the other interesting info     
for column in sub_hc_info.columns[1:]:
    df[column] = []
# add the column name for correlation values
df = pd.concat((df, df_corr), axis = 1)

In [13]:
df

,sub,scanner,group,sex,age,ymrs,mrt_idsc,Middle_temporal_gyrus_RH_X_Ventromedial_prefrontal_cortex,Posterior_cingulate_cortex_X_Ventromedial_prefrontal_cortex,Posterior_cingulate_cortex_X_Middle_temporal_gyrus_RH,...,Hippocampus_anterior_RH_X__Pars_triangularis_RH,Hippocampus_anterior_RH_X__Superior_frontal_sulcus_anterior_LH2,Hippocampus_anterior_RH_X__Superior_frontal_sulcus_anterior_RH,Hippocampus_anterior_RH_X__Angular_gyrus_inferior_LH,Hippocampus_anterior_RH_X__Angular_gyrus_inferior_RH,Hippocampus_anterior_RH_X__Amygdala_LH,Hippocampus_anterior_RH_X__Amygdala_RH,Hippocampus_anterior_RH_X__Hippocampus_posterior_LH,Hippocampus_anterior_RH_X__Hippocampus_posterior_RH,Hippocampus_anterior_RH_X__Hippocampus_anterior_LH


## Adding all the relevant info from every subject into the dataframe

In [15]:
for sub in sub_idx:
    
    #print(sub)
    
    if sub in sub_excluded: # skip the excluded participants
        continue
    
    else: 
    
        # create a list which will be filled with alle the infos 
        sub_list = [] 

        # add the subjects name to the list 
        sub_list.append(sub)
            
        # add scanner/ site
        if 'B' == sub[0]: #Berlin
            sub_list.append('B')
        if 'O' == sub[0]: #Bochum
            sub_list.append('BO')
        if 'D' == sub[0]: #Dresden
            sub_list.append('DD')
        if 'F' == sub[0]: #Frankfurt
            sub_list.append('F')
        if 'G' == sub[0]: #Goettingen
            sub_list.append('GOE')
        if 'H' == sub[0]: #Hamburg
            sub_list.append('HH')
        if 'M' == sub[0]: #Marburg
            sub_list.append('MR')
        if 'T' == sub[0]: #Tübingen
            sub_list.append('TUE')
        
        # add group and all other info except the correlation values 
        if sub in sub_hc:
            # add group 
            sub_list.append('1') #HC
            # add further info 
            idx = sub_hc_info.index[sub_hc_info['BIDS'] == sub][0]
            info = sub_hc_info.iloc[idx].tolist()
            for single_info in info[1:]:
                sub_list.append(single_info)
            
        if sub in sub_nr:
            # add group 
            sub_list.append('2') #NR
            # add further info 
            idx = sub_nr_info.index[sub_nr_info['BIDS'] == sub][0]
            info = sub_nr_info.iloc[idx].tolist()
            for single_info in info[1:]:
                sub_list.append(single_info)
            
        if sub in sub_lr:
            # add group 
            sub_list.append('3') #LR
            # add further info 
            idx = sub_lr_info.index[sub_lr_info['BIDS'] == sub][0]
            info = sub_lr_info.iloc[idx].tolist()
            for single_info in info[1:]:
                sub_list.append(single_info)
            
        if sub in sub_hr:
            # add group 
            sub_list.append('4') #HR
            # add further info 
            idx = sub_hr_info.index[sub_hr_info['BIDS'] == sub][0]
            info = sub_hr_info.iloc[idx].tolist()
            for single_info in info[1:]:
                sub_list.append(single_info)
            
        if sub in sub_bd:
            # add group 
            sub_list.append('5') #BD
            # add further info 
            idx = sub_bd_info.index[sub_bd_info['BIDS'] == sub][0]
            info = sub_bd_info.iloc[idx].tolist()
            for single_info in info[1:]:
                sub_list.append(single_info)

        # add all connectivity values via accessing the corresponding correlation vector
        dir_matrices_derivatives = f'/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-partial_correlation/sub-{sub}/2903/sub-{sub}_atlas-difumo_kind-partial_correlation_connectivity_vector_2903.npy'
        sub_vector = np.load(dir_matrices_derivatives)
        # every value on its own will be added 
        for value in sub_vector:
            sub_list.append(value)
        
        df.loc[len(df)] = sub_list

In [16]:
#df['group'] = df['group'].astype('category')
df['sex'] = df['sex'].astype('category')
df['scanner'] = df['scanner'].astype('category')

# changing the column name, certain entries and the data type of the idsc results 
df = df.rename(columns={'mrt_idsc':'idsc'})
df['idsc'] = df['idsc'].replace('8 (not complete)',np.NaN)
df['idsc'] = df['idsc'].replace('2 (not complete)',np.NaN)

In [17]:
# to get rid of empty spaces within the region naming
df.columns = df.columns.str.replace('-','_')

In [18]:
df

,sub,scanner,group,sex,age,ymrs,idsc,Middle_temporal_gyrus_RH_X_Ventromedial_prefrontal_cortex,Posterior_cingulate_cortex_X_Ventromedial_prefrontal_cortex,Posterior_cingulate_cortex_X_Middle_temporal_gyrus_RH,...,Hippocampus_anterior_RH_X__Pars_triangularis_RH,Hippocampus_anterior_RH_X__Superior_frontal_sulcus_anterior_LH2,Hippocampus_anterior_RH_X__Superior_frontal_sulcus_anterior_RH,Hippocampus_anterior_RH_X__Angular_gyrus_inferior_LH,Hippocampus_anterior_RH_X__Angular_gyrus_inferior_RH,Hippocampus_anterior_RH_X__Amygdala_LH,Hippocampus_anterior_RH_X__Amygdala_RH,Hippocampus_anterior_RH_X__Hippocampus_posterior_LH,Hippocampus_anterior_RH_X__Hippocampus_posterior_RH,Hippocampus_anterior_RH_X__Hippocampus_anterior_LH
0,BC01,B,1,male,29.0,0.0,7.0,0.100643,0.091842,0.017214,...,0.154628,-0.146482,0.026668,0.258933,-0.243835,-0.042882,0.012358,0.205664,-0.099312,0.189752
1,BC02,B,1,male,29.0,0.0,4.0,-0.004687,0.193546,0.009188,...,0.101544,0.191384,-0.029808,-0.031635,0.274054,0.058302,-0.006439,0.050933,0.154651,0.333206
2,BC03,B,1,female,20.0,0.0,7.0,0.025617,-0.153666,0.074608,...,0.024870,-0.246607,-0.028187,-0.071547,0.221382,0.187747,0.109672,-0.027314,0.319234,0.302386
3,BC04,B,1,female,25.0,0.0,8.0,0.266660,0.052269,0.143487,...,-0.004562,-0.076414,0.155846,0.094810,0.262270,-0.084050,0.013704,0.328977,0.158299,0.413792
4,BC05,B,1,female,21.0,0.0,4.0,-0.263986,-0.087842,-0.090361,...,-0.017641,0.160246,-0.237093,0.049388,-0.084516,0.143440,-0.026665,0.020328,0.047392,0.294934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,TR02,TUE,2,female,22.0,0.0,15.0,0.036970,0.065108,0.188403,...,0.053108,0.147527,0.151138,-0.272490,-0.105028,0.038338,-0.073904,-0.074723,0.018781,0.233328
356,TR03,TUE,3,male,23.0,4.0,14.0,-0.094007,0.020915,0.139322,...,0.080729,0.055815,-0.124827,-0.158031,0.078489,0.003749,-0.166596,-0.063737,0.004317,0.751097
357,TS03,TUE,5,female,20.0,0.0,8.0,-0.113868,0.226948,0.203391,...,0.131485,-0.002585,0.061780,0.065400,-0.002834,0.029733,0.186177,0.028419,0.149475,0.204508
358,TS04,TUE,5,male,26.0,2.0,0.0,0.085807,-0.139296,0.176873,...,0.093797,0.155410,0.089063,0.055008,-0.195665,-0.127932,-0.219593,0.061078,0.217678,0.408282


In [19]:
df.to_csv('/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/stattable_partialcorrelation_final.tsv', sep='\t', index=False)  

In [ ]:
### z transformation per row in regards to mean and standard deviation
#import scipy.stats as stats
#transformed_df = df.iloc[:,7:].apply(stats.zscore)
#df.update(transformed_df)

#df.to_csv('/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/stattable_zscores_2004.tsv', sep='\t', index=False)  